<a href="https://colab.research.google.com/github/leohafsa/ML_Unets_data/blob/main/MaxSamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install scipy==1.1.0
! pip install pillow

     |████████████████████████████████| 31.2MB 151kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [2]:
!pip install SimpleITK

     |████████████████████████████████| 44.9MB 100kB/s 


In [3]:
cd /content/drive/My Drive/unidata/StrokeSegmentation-master

/content/drive/My Drive/unidata/StrokeSegmentation-master


In [4]:
! python metrics.py

2020-11-26 15:32:53.499443: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [5]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 15 17:18:38 2017

@author: Inom Mirzaev
"""

from __future__ import division, print_function
from collections import defaultdict
import os, pickle, sys
import shutil
from functools import partial


import numpy as np
from scipy.misc import imresize
from skimage.transform import resize
from skimage.exposure import equalize_adapthist, equalize_hist
import pandas as pd
import random
import SimpleITK as sitk
import scipy.io as scio
from skimage import measure

root_path='/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/'

#T2_list=['031948','031950','031957','031959','031972']

def Coundt_file_number(input_path):
    FileNum=len([lists for lists in os.listdir(input_path)])
    return FileNum

def PreProcessing():
     #for siteidx in range(9):
         #site_path=root_path+'Site'+str(siteidx+1)
         #files = os.listdir(site_path)
         #print(files)
         #for file in files:
             #input_path=site_path+'//'+file+'//t01//'
             #print(input_path)
      for siteidx in range(9):
         site_path=root_path+'Site'+str(siteidx+1)
         files = os.listdir(site_path)
         print(files)
         for file in files:
             input_path=site_path+'//'+file+'//t01//'
             print(input_path)
             FileNum=Coundt_file_number(input_path)
             img_itk = sitk.ReadImage(input_path+file+'_t1w_deface_stx.nii.gz')
             img_array = sitk.GetArrayFromImage(img_itk)
            
             gs_itk = sitk.ReadImage(input_path+file+'_LesionSmooth_stx.nii.gz')
             gs_array = sitk.GetArrayFromImage(gs_itk)
             gs_array[gs_array>0.5]=1
             if FileNum>2:
                 Extra_lesion=FileNum-2
                 for i in range(Extra_lesion):
                     Extra_itk = sitk.ReadImage(input_path+file+'_LesionSmooth_'+str(i+1)+'_stx.nii.gz')
                     Extra_array = sitk.GetArrayFromImage(Extra_itk)
                     gs_array[Extra_array>0.5]=1

             itkimage = sitk.GetImageFromArray(img_array, isVector=False)
             itkimage.SetSpacing(img_itk.GetSpacing())
             itkimage.SetOrigin(img_itk.GetOrigin())
             sitk.WriteImage(itkimage, '/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image/'+ str(file) + '_IMGAG.nii', True)     

             itkimage = sitk.GetImageFromArray(gs_array, isVector=False)
             itkimage.SetSpacing(img_itk.GetSpacing())
             itkimage.SetOrigin(img_itk.GetOrigin())
             sitk.WriteImage(itkimage, '/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Label/'+ str(file) + '_LABEL.nii', True)     

if __name__=='__main__':
    PreProcessing() 
  

[]
['031854', '031848', '031851', '031846', '031847', '031844', '031843', '031836', '031841', '031837', '031835', '031834', '031830', '031826', '031832', '031828', '031831', '031829', '031827', '031824', '031823']
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site2//031854//t01//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site2//031848//t01//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site2//031851//t01//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site2//031846//t01//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site2//031847//t01//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site2//031844//t01//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site2//031843//t01//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site2//031836//t01//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1

In [6]:
'''
data generator
ATLAS dataset has been transformed into .h5 format
'''

import numpy as np
import os
import cv2
from SimpleITK  import *
import scipy.io as scio
from metrics import *
import SimpleITK as sitk
from skimage.exposure import equalize_adapthist, equalize_hist


# 189,233,197,
# [], 192[21:213],160[19:179]

def SBS_equalize_adapthist(imgs):

    new_imgs = np.zeros([len(imgs), imgs.shape[1], imgs.shape[2]])
    for mm, img in enumerate(imgs):
        img = equalize_adapthist( img, clip_limit=0.05 )
        new_imgs[mm] = img
    return new_imgs
    
    

Mean_Stroke=46.26
STD_Stroke=21.42

def data_to_array(data_path,code_path,current_fold,FD,Mean_Stroke,STD_Stroke):
    images_z = []
    labels_z= []
    imlb_list = os.listdir(data_path+'Image')
    imlb_list.sort()
    num_patient_train=len(imlb_list)
    #num_patient_train=5
    print(num_patient_train)
    for l in range(num_patient_train):
        if l % FD ==current_fold:
            continue
        else:
            file_im=imlb_list[l]
            #print(file_im.find('_'))
            idx=file_im[0:file_im.find('_')]
            print(idx)
            image_path =data_path+'Image\\'+file_im
            label_path =data_path+'Label\\'+idx+'_LABEL.nii'

            label_itk = sitk.ReadImage(label_path)
            label_array = sitk.GetArrayFromImage(label_itk)
            #print(label_itk.GetSpacing())
        
            label_array[label_array>0.5]=1
            image_array = itk_to_array(image_path)

            # 
            #image_array= (image_array - Mean_Stroke) / float(STD_Stroke)
            Min_z,Max_z,Min_x,Max_x,Min_y,Max_y=Find_ranges(label_array,0,0)
        
            Crop_image=image_array[Min_z:Max_z,21:213,19:179]
            Crop_label=label_array[Min_z:Max_z,21:213,19:179]
            print(np.shape(Crop_label))
            #Crop_image=Crop_image/50 - 1
            #Crop_image=SBS_equalize_adapthist(Crop_image)
            images_z.append(Crop_image)
            labels_z.append(Crop_label)
      
    images_z = np.concatenate(images_z, axis=0).reshape(-1, 192, 160, 1)
    labels_z = np.concatenate(labels_z, axis=0).reshape(-1, 192, 160, 1)
    print(np.shape(labels_z))

   
    np.save(code_path+'NPY_data\\X_FD'+str(current_fold)+'.npy', images_z)
    np.save(code_path+'NPY_data\\y_FD'+str(current_fold)+'.npy', labels_z)
'''
#Mask
def data_to_array_with_MASK(data_path,code_path,current_fold,FD):
    images_z = []
    labels_z= []
    masks_z= []
    images_y = []
    labels_y= []
    masks_y= []
    images_x = []
    labels_x= []
    masks_x= []

    imlb_list = os.listdir(data_path+'Image')
    imlb_list.sort()
    num_patient_train=len(imlb_list)
   # num_patient_train=5
    print(num_patient_train)
    for l in range(num_patient_train):
        if l % FD ==current_fold:
            continue
        else:
            file_im=imlb_list[l]
            print(file_im)
            idx=file_im[0:file_im.find('_')]
            print(idx)
            image_path =data_path+'Image\\'+file_im
            label_path =data_path+'Label\\'+idx+'_LABEL.nii'
            Mask_path =data_path+'Mask\\'+idx+'_REG.nii'

            label_itk = sitk.ReadImage(label_path)
            label_array = sitk.GetArrayFromImage(label_itk)
        
            label_array[label_array>0.5]=1
            image_array = itk_to_array(image_path)
            Mask_array = itk_to_array(Mask_path)

            
            #print(Mask_array.shape) 189, 233,197 # S A S 
            Min_z,Max_z,Min_x,Max_x,Min_y,Max_y=Find_ranges(label_array,0,0)
        
            Crop_image=image_array[Min_z:Max_z,21:213,19:179]
            Crop_label=label_array[Min_z:Max_z,21:213,19:179]
            Crop_Mask=Mask_array[Min_z:Max_z,21:213,19:179]
            images_z.append(Crop_image)
            labels_z.append(Crop_label)
            masks_z.append(Crop_Mask)

            Crop_image=image_array[7:167,Min_x:Max_x,19:179]
            Crop_label=label_array[7:167,Min_x:Max_x,19:179]
            Crop_Mask=Mask_array[7:167,Min_x:Max_x,19:179]
            Crop_image=np.transpose(Crop_image,[1,0,2])
            Crop_label=np.transpose(Crop_label,[1,0,2])
            Crop_Mask=np.transpose(Crop_Mask,[1,0,2])
            images_x.append(Crop_image)
            labels_x.append(Crop_label)
            masks_x.append(Crop_Mask)
            print(Crop_image.shape)

            Crop_image=image_array[7:167,21:213,Min_y:Max_y]
            Crop_label=label_array[7:167,21:213,Min_y:Max_y]
            Crop_Mask=Mask_array[7:167,21:213,Min_y:Max_y]
            Crop_image=np.transpose(Crop_image,[2,1,0])
            Crop_label=np.transpose(Crop_label,[2,1,0])
            Crop_Mask=np.transpose(Crop_Mask,[2,1,0])
            images_y.append(Crop_image)
            labels_y.append(Crop_label)
            masks_y.append(Crop_Mask)
   
    images_z = np.concatenate(images_z, axis=0).reshape(-1, 192, 160, 1)
    labels_z = np.concatenate(labels_z, axis=0).reshape(-1, 192, 160, 1)
    masks_z = np.concatenate(masks_z, axis=0).reshape(-1, 192, 160, 1)

    images_x = np.concatenate(images_x, axis=0).reshape(-1, 160, 160, 1)
    labels_x = np.concatenate(labels_x, axis=0).reshape(-1, 160, 160, 1)
    masks_x = np.concatenate(masks_x, axis=0).reshape(-1, 160, 160, 1)

    images_y = np.concatenate(images_y, axis=0).reshape(-1, 192, 160, 1)
    labels_y = np.concatenate(labels_y, axis=0).reshape(-1, 192, 160, 1)
    masks_y = np.concatenate(masks_y, axis=0).reshape(-1, 192, 160, 1)
    print(np.shape(labels_z))

    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Trans_X_FD'+str(current_fold)+'.mat', {'images':images_z})
    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Trans_y_FD'+str(current_fold)+'.mat', {'labels':labels_z})
    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Trans_M_FD'+str(current_fold)+'.mat', {'masks':masks_z})
    np.save(code_path+'NPY_data\\Trans_X_FD'+str(current_fold)+'.npy', images_z)
    np.save(code_path+'NPY_data\\Trans_y_FD'+str(current_fold)+'.npy', labels_z)
    np.save(code_path+'NPY_data\\Trans_M_FD'+str(current_fold)+'.npy', masks_z)

    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Sagit_X_FD'+str(current_fold)+'.mat', {'images':images_x})
    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Sagit_y_FD'+str(current_fold)+'.mat', {'labels':labels_x})
    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Sagit_M_FD'+str(current_fold)+'.mat', {'masks':masks_x})
    np.save(code_path+'NPY_data\\Sagit_X_FD'+str(current_fold)+'.npy', images_x)
    np.save(code_path+'NPY_data\\Sagit_y_FD'+str(current_fold)+'.npy', labels_x)
    np.save(code_path+'NPY_data\\Sagit_M_FD'+str(current_fold)+'.npy', masks_x)

    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Coron_X_FD'+str(current_fold)+'.mat', {'images':images_y})
    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Coron_y_FD'+str(current_fold)+'.mat', {'labels':labels_y})
    #scio.savemat('H:\\ATLAStoke\\code_matlab\\Coron_M_FD'+str(current_fold)+'.mat', {'masks':masks_y})
    np.save(code_path+'NPY_data\\Coron_X_FD'+str(current_fold)+'.npy', images_y)
    np.save(code_path+'NPY_data\\Coron_y_FD'+str(current_fold)+'.npy', labels_y)
    np.save(code_path+'NPY_data\\Coron_M_FD'+str(current_fold)+'.npy', masks_y)
'''
FD=5
#Mask

def Find_lesion(mask,axis=0):
    if axis==0:
        max_axis0=np.zeros([mask.shape[0],1])
        for i in range(mask.shape[0]):
            #print(np.max(mask[i,:,:]))
            max_axis0[i]=np.max(mask[i,:,:])
        #print(max_axis0.shape)
        arr=np.nonzero(max_axis0)
        #print(arr)
    elif axis==1:
        max_axis1=np.zeros([mask.shape[1],1])
        for i in range(mask.shape[1]):
            #print(np.max(mask[:,i,:]))
            max_axis1[i]=np.max(mask[:,i,:])
        arr=np.nonzero(max_axis1)
    elif axis==2:
        max_axis2=np.zeros([mask.shape[2],1])
        for i in range(mask.shape[2]):
            #print(np.max(mask[:,:,i]))
            max_axis2[i]=np.max(mask[:,:,i])
        #print(max_axis0.shape)
        arr=np.nonzero(max_axis2)
        #print(arr)
    return arr[0]

def data_to_array_with_MASK(data_path,code_path,current_fold,FD):
    images_z = []
    labels_z= []
    masks_z= []
    images_y = []
    labels_y= []
    masks_y= []
    images_x = []
    labels_x= []
    masks_x= []

    imlb_list = os.listdir(data_path+'Image')
    imlb_list.sort()
    num_patient_train=len(imlb_list)
    #num_patient_train=5
    print(num_patient_train)
    for l in range(num_patient_train):
        if l % FD ==current_fold:
            continue
        else:
            file_im=imlb_list[l]
            print(file_im)
            idx=file_im[0:file_im.find('_')]
            print(idx)
            image_path =data_path+'Image//'+file_im
            label_path =data_path+'Label//'+idx+'_LABEL.nii'
            Mask_path =data_path+'Mask//'+idx+'_REG.nii'
            print(image_path)
            print(Mask_path)
            print(idx)

            label_itk = sitk.ReadImage(label_path)
            label_array = sitk.GetArrayFromImage(label_itk)
        
            label_array[label_array>0.5]=1

            image_itk = sitk.ReadImage(image_path)
            image_array = sitk.GetArrayFromImage(image_itk)

            mask_itk = sitk.ReadImage(Mask_path)
            Mask_array = sitk.GetArrayFromImage(mask_itk)
            #Mask_array = mask_itk(Mask_path)
             
          
        
            #image_array = itk_to_array(image_path)
            #Mask_array = itk_to_array(Mask_path)
            '''
            arr0=Find_lesion(label_array,0)
            arr1=Find_lesion(label_array,1)
            arr2=Find_lesion(label_array,2)
        
            Crop_image=image_array[arr0,:,:]
            Crop_label=label_array[arr0,:,:]
            Crop_Mask=Mask_array[arr0,:,:]
            images_z.append(Crop_image)
            labels_z.append(Crop_label)
            masks_z.append(Crop_Mask)
            #print(Crop_image.shape)

            Crop_image=image_array[:,arr1,:]
            Crop_label=label_array[:,arr1,:]
            Crop_Mask=Mask_array[:,arr1,:]
            Crop_image=np.transpose(Crop_image,[1,0,2])
            Crop_label=np.transpose(Crop_label,[1,0,2])
            Crop_Mask=np.transpose(Crop_Mask,[1,0,2])
            images_x.append(Crop_image)
            labels_x.append(Crop_label)
            masks_x.append(Crop_Mask)

            Crop_image=image_array[:,:,arr2]
            Crop_label=label_array[:,:,arr2]
            Crop_Mask=Mask_array[:,:,arr2]
            Crop_image=np.transpose(Crop_image,[2,1,0])
            Crop_label=np.transpose(Crop_label,[2,1,0])
            Crop_Mask=np.transpose(Crop_Mask,[2,1,0])
            images_y.append(Crop_image)
            labels_y.append(Crop_label)
            masks_y.append(Crop_Mask)
            #print(Crop_image.shape)
            '''
            
            Min_z,Max_z,Min_x,Max_x,Min_y,Max_y=Find_ranges(label_array ,0,0)
            Crop_image=image_array[Min_z:Max_z,:,:]
            Crop_label=label_array[Min_z:Max_z,:,:]
            Crop_Mask=Mask_array[Min_z:Max_z,:,:]
            images_z.append(Crop_image)
            labels_z.append(Crop_label)
            masks_z.append(Crop_Mask)
            #print(Crop_image.shape)

            Crop_image=image_array[:,Min_x:Max_x,:]
            Crop_label=label_array[:,Min_x:Max_x,:]
            Crop_Mask=Mask_array[:,Min_x:Max_x,:]
            Crop_image=np.transpose(Crop_image,[1,0,2])
            Crop_label=np.transpose(Crop_label,[1,0,2])
            Crop_Mask=np.transpose(Crop_Mask,[1,0,2])
            images_x.append(Crop_image)
            labels_x.append(Crop_label)
            masks_x.append(Crop_Mask)

            Crop_image=image_array[:,:,Min_y:Max_y]
            Crop_label=label_array[:,:,Min_y:Max_y]
            Crop_Mask=Mask_array[:,:,Min_y:Max_y]
            Crop_image=np.transpose(Crop_image,[2,1,0])
            Crop_label=np.transpose(Crop_label,[2,1,0])
            Crop_Mask=np.transpose(Crop_Mask,[2,1,0])
            images_y.append(Crop_image)
            labels_y.append(Crop_label)
            masks_y.append(Crop_Mask)

   
    images_z = np.concatenate(images_z, axis=0).reshape(-1, 233,197, 1)
    labels_z = np.concatenate(labels_z, axis=0).reshape(-1, 233,197, 1)
    masks_z = np.concatenate(masks_z, axis=0).reshape(-1, 233, 197, 1)

    #images_x = np.concatenate(images_x, axis=0).reshape(-1, 189,197, 1)
    #labels_x = np.concatenate(labels_x, axis=0).reshape(-1, 189,197, 1)
    #masks_x = np.concatenate(masks_x, axis=0).reshape(-1, 189,197, 1)

    images_y = np.concatenate(images_y, axis=0).reshape(-1, 233, 189, 1)
    labels_y = np.concatenate(labels_y, axis=0).reshape(-1, 233, 189, 1)
    masks_y = np.concatenate(masks_y, axis=0).reshape(-1,233, 189, 1)
    print(np.shape(labels_z))

  
    np.save(code_path+'NPY_data\\Trans_X_FD'+str(current_fold)+'.npy', images_z)
    np.save(code_path+'NPY_data\\Trans_y_FD'+str(current_fold)+'.npy', labels_z)
    np.save(code_path+'NPY_data\\Trans_M_FD'+str(current_fold)+'.npy', masks_z)

   
    np.save(code_path+'NPY_data\\Coron_X_FD'+str(current_fold)+'.npy', images_y)
    np.save(code_path+'NPY_data\\Coron_y_FD'+str(current_fold)+'.npy', labels_y)
    np.save(code_path+'NPY_data\\Coron_M_FD'+str(current_fold)+'.npy', masks_y)

current_fold=0
data_path = '/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/'
code_path = '/content/drive/My Drive/unidata/StrokeSegmentation-master/code'
data_to_array_with_MASK(data_path,code_path,current_fold,FD)


114
031824_IMGAG.nii
031824
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031824_IMGAG.nii
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Mask//031824_REG.nii
031824
031826_IMGAG.nii
031826
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031826_IMGAG.nii
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Mask//031826_REG.nii
031826
031827_IMGAG.nii
031827
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031827_IMGAG.nii
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Mask//031827_REG.nii
031827
031828_IMGAG.nii
031828
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031828_IMGAG.nii
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Mask//031828_REG.nii
031828
031830_IMGAG.nii
031830
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031830_IMGAG.nii
/content/drive/My Drive/unidata/StrokeSegmentation

In [7]:
! python loss.py

2020-11-26 16:23:40.457127: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [8]:
! python model_Unet_pading.py

2020-11-26 16:23:52.973603: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-26 16:23:54.797877: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-26 16:23:54.854283: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-11-26 16:23:54.854355: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (e18de9931ff4): /proc/driver/nvidia/version does not exist
2020-11-26 16:23:54.883887: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2300000000 Hz
2020-11-26 16:23:54.884333: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e8aa00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-26 16:23:54.884405: I tensorflow/com

In [ ]:
! python train_Unet_main.py

2020-11-26 16:23:58.196473: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
114
031824_IMGAG.nii
031824
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031824_IMGAG.nii
031826_IMGAG.nii
031826
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031826_IMGAG.nii
031827_IMGAG.nii
031827
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031827_IMGAG.nii
031828_IMGAG.nii
031828
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031828_IMGAG.nii
031830_IMGAG.nii
031830
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031830_IMGAG.nii
031831_IMGAG.nii
031831
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031831_IMGAG.nii
031832_IMGAG.nii
031832
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031832_IMGAG.nii
031834_IMGAG.nii
031834
/content/drive/My Drive/unid

In [10]:
! python test_Unet_main.py

2020-11-26 17:09:35.395027: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "test_Unet_main.py", line 13, in <module>
    from keras.optimizers import Adam
  File "/usr/local/lib/python3.6/dist-packages/keras/__init__.py", line 3, in <module>
    from tensorflow.keras.layers.experimental.preprocessing import RandomRotation
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 41, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/__init__.py", line 40, in <module>
    from tensorflow.python.eager import context
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/context.py", line 35, in <module>
    from tensorflow.python import pywrap_tfe
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tfe.py", line 28, in <module>
  